In [29]:
# Note: Original saved in Tahir Sproj folder

# Deep Learning Libraries
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

# Data Manipulation and Analysis
import numpy as np
# import pandas as pd

# Data Visualization
import matplotlib
import matplotlib.pyplot as plt

# File and System Interaction
import os
from pathlib import Path
import torch.optim as optim


# Date and Time Handling
import time
import datetime

# Neural Architecture
try:
    from torchinfo import summary
except:
    # %pip install torchinfo
    from torchinfo import summary
from scipy.fftpack import dct

In [40]:
%load_ext autoreload
%autoreload 2

# from python_scripts import dataset_processing
# from python_scripts import architecture
# from python_scripts import revised_architecture
# from python_scripts import training
# from python_scripts import logs_and_results
# from python_scripts import img_pdf_compiler
from python_scripts import utils
from python_scripts import DUST_MC
from python_scripts import dustmc_unrolled
from python_scripts import test
from python_scripts import generate_synthetic_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
# Example usage:
model = dustmc_unrolled.DustNet(K=5, mu=0, sigma=1, m=3, n=3, d=3, T=10)
S = torch.randn(3, 10)
output = model(S)
print(output)

tensor([[ 889.3987, -400.7791,  701.3297, 1246.3589,  -66.9305, -652.9890,
         -238.3193,  -70.9368,  654.4198,  337.0663],
        [ 230.4068, -349.2725,   67.1426,  317.9235,  -43.7342, -118.5419,
          -41.4297,  -64.4256,  144.0345,  286.7173],
        [ 819.9326, -348.4921,  563.7321, 1431.6442, -156.5717, -512.8481,
         -189.2724,  -44.1213,  656.7279,  340.4797]], grad_fn=<MmBackward0>)


In [37]:
import torch

# Example values
DH = torch.randn(10, 5)  # Shape (n, T)

# Compute βi values
beta_i = torch.exp(-0.5 * torch.sum(DH ** 2, dim=0))  # Shape (T,)

# Ensure indexing returns a tensor
index = 2
indexed_value = beta_i[index]

print(f"beta_i: {beta_i}")
print(f"Indexed value: {indexed_value}")
print(f"Type of indexed value: {type(indexed_value)}")  # Should be <class 'torch.Tensor'>


beta_i: tensor([0.0045, 0.0176, 0.0199, 0.0003, 0.0161])
Indexed value: 0.01992465741932392
Type of indexed value: <class 'torch.Tensor'>


In [4]:
# Get relevant arguements for DUSTMC

M_train, M_Omega_train, M_test, M_Omega_test = generate_synthetic_data.generate_simple_gaussian_noise(150, 300, 10, 10, 2, 0.5, 3)

In [29]:
# Set the random seed for reproducibility
# np.random.seed(42)

rand_batch_idx = np.random.randint(0, M_Omega_train.shape[0])
X = M_Omega_train[rand_batch_idx, :, :]

# Initialize a m x n A measurement sensing matrix with random values from a uniform distribution between 0 and 1
A = np.random.uniform(0, 1, (100, 150)) # A is m x n. Since m << n and n is normally 150

# Initalize the Dictonary D which is n x d and d >> n by DCT. d is taken as 512
random_matrix = np.random.uniform(0, 1, (150, 512))

# Apply the Discrete Cosine Transform (DCT) to the matrix
D = dct(random_matrix, type = 2, norm = 'ortho')

# Initialize K (no. of iterations), and hyperparameters lambda_1, lambda_2, and c
K = 50
lambda_1 = 0.1
lambda_2 = 0.4
c = 1

In [31]:
# Now run the attention based algo on X to get its recovered/reconstructed form S
S = test.attention_based_algo(A, D, X, K, lambda_1, lambda_2, c)

C:\Users\Talha\AppData\Local\Temp\ipykernel_20176\157688921.py:35: RuntimeWarning: overflow encountered in exp
  exp_values = np.exp(exponent_terms)
C:\Users\Talha\AppData\Local\Temp\ipykernel_20176\157688921.py:38: RuntimeWarning: invalid value encountered in multiply
  G_numerator = np.sum(beta_values[:, None] * exp_values[:, None] * H.T, axis=0)
C:\Users\Talha\AppData\Local\Temp\ipykernel_20176\157688921.py:39: RuntimeWarning: invalid value encountered in multiply
  G_denominator = np.sum(beta_values * exp_values)


In [ ]:
# Now measure loss between reconstructed and groundtruth

groundtruth = M_train[rand_batch_idx, :, :]

print(f'The squared L2 norm loss is: {utils.compute_squared_l2_norm_loss(S, groundtruth)}')